This is my solution to assignment 1 of the WiDS Project : Learning with quantum computers. The link of the problem statement is [this](https://github.com/siddhant-midha/WiDS-22-Learning-with-quantum-computers-/blob/main/problems/set1.md)

# **Quick installation commands**

In [ ]:
!pip install qiskit
!pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src
!pip install pylatexenc

# **Importing libraries**

In [1]:
# Importing numpy
import numpy as np

# Analysis tools
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram, plot_bloch_vector
from math import sqrt, pi

## **Oracles**

Before we can even begin to adress the Deutsch-Jozsa problem, we need to construct the black-box functions or the "oracles" which are unknown to the algorithm we will design in order to solve the problem.

# *Constructing the constant oracle*

In [2]:
class constant_oracle : 
    
    def __init__(self, n) : 
        self.n = n
        self.output = np.random.randint(2)
        self.draw("0"*n)

    def draw(self, input) : 
        '''
        Returns a draw from the constant oracle
        '''
        self.const_oracle = QuantumCircuit(self.n+1, self.n+1)
        
        if self.output == 1:
            self.const_oracle.x(self.n)
        self.const_oracle.measure(self.n,0)

        sim = Aer.get_backend('aer_simulator') 
        result = sim.run(self.const_oracle).result()
        return list(result.get_counts().keys())[0]


# *Constructing the balanced oracle*

For this we use the simple fact that the xor of uniform random distributions is jsut another uniform random distribution. Hence, taking xor of all bits is enough to obtain a balanced oracle

In [3]:
class balanced_oracle : 

    def __init__(self, n) : 
        self.n = n
        self.draw("0"*n)
        

    def draw(self, input) : 
        '''
        Returns a draw from the balanced oracle
        '''
        self.balanced_oracle = QuantumCircuit(self.n+1, self.n+1)

        # Place X-gates
        for qubit in range(len(input)):
            if input[qubit] == '1':
                self.balanced_oracle.x(qubit)

        # Use barrier as divider
        self.balanced_oracle.barrier()

        # Controlled-NOT gates
        for qubit in range(self.n):
            self.balanced_oracle.cx(qubit, self.n)

        self.balanced_oracle.barrier()

        # Place X-gates
        for qubit in range(len(input)):
            if input[qubit] == '1':
                self.balanced_oracle.x(qubit)

        self.balanced_oracle.measure(self.n, 0)

        sim = Aer.get_backend('aer_simulator') 
        result = sim.run(self.balanced_oracle).result()
        return list(result.get_counts().keys())[0]
    




# *Running both oracles*

**Constant Oracle**

In [17]:
oracle = constant_oracle(3)
oracle.draw("101")

'0001'

**Balanced oracle**

In [7]:
oracle = balanced_oracle(3)
oracle.draw("101")

'0000'

# **Part 1 : Deterministic Algorithm**

In this section, I shall present a deterministic algorithm to solve the Deutsch-Jozsa problem. The algorithm relies upon querying the previously defined black boxes $ 2^{n-1} + 1$ times. (where n is the number of input bits)

In [10]:
def classical_algo(f, n) : 
    '''
    Input : A function f, number of bits in the input n (natural number)
    Returns : A boolean (true if the function is balanced)
    '''
    header = "0"*n
    val = f(header)
    for i in range(1, 2**(n-1) + 2) :
        num = bin(i).replace("0b", "")
        input = (header + num)[-n:] 
        if f(input) != val : 
            return True

    return False

Now, we shall test this algorithm for both cases namely : 
* Constant oracle
* Balanced oracle
Taking functions $f : \{0, 1\}^3 \to \{0, 1\}$

In [11]:
# Number of input bits 
N = 3

# First, constructing a constant oracle
oracle = constant_oracle(N)
if classical_algo(oracle.draw, N) : print("Balanced oracle")
else : print("Constant oracle")


Constant oracle


In [12]:
# Next, constructing a balanced oracle
oracle = balanced_oracle(N)
if classical_algo(oracle.draw, N) : print("Balanced oracle")
else : print("Constant oracle")

Balanced oracle


Hence, we are done with the classical deterministic algorithm for this problem.

# **Part 2 : Algorithm using Quantum circuits**

Next, we shall implement the Deutsch-Jozsa algorithm as presented [here](https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html)

In [13]:
from numpy.lib.twodim_base import triu_indices_from
def dj_algo(oracle, n) :
    '''
    Input : A function f, number of bits in the input n (natural number)
    Returns : A boolean (true if the function is balanced)
    ''' 

    dj_ckt = QuantumCircuit(n+1, n+1)  
    # The last bit should be a 1
    dj_ckt.x(n)
    for i in range(n+1) : 
      dj_ckt.h(i)

    dj_ckt.compose(oracle, inplace = True)

    # Applying H gates again
    for i in range(n) : 
      dj_ckt.h(i)

    # Measuring the first n qubits
    for i in range(n) : 
      dj_ckt.measure(i, i+1)
    
    sim = Aer.get_backend('aer_simulator') 
    result = sim.run(dj_ckt).result()
    keys = list(result.get_counts().keys())

    if "0" + "0"*n in keys : 
        return False
    elif "1" + "0"*n in keys : 
        return False
    else : 
        return True

Now, let's test this for both algorithms

In [14]:
oracle = balanced_oracle(N).balanced_oracle
result = dj_algo(oracle, N)

if result : print("Balanced oracle")
else : print("Constant oracle")

Balanced oracle


In [15]:
oracle = constant_oracle(N).const_oracle
oracle.draw()
result = dj_algo(oracle, N)

if result : print("Balanced oracle")
else : print("Constant oracle")

Constant oracle


Hence, we see that the deutch-josza algorithm also computes the results correctly.

# **Part 3 : Randomised Classical Algorithm**

In this section, we shall make random draws from the domain space and compute the probability of the oracle being balanced or constant. Now, let us compute the probabilities.
\begin{equation}
P(balanced|data) = \frac{P(balanced, data)}{P(data)} \\ 
\end{equation}
If any two output bits are different, we are done. If not, let us calculate the probability of that happening for $n$ draws.
\begin{equation}
P(00{}_{\cdots}0) = \frac{1}{2}.\frac{1}{2} + \frac{1}{2}.\frac{1}{2^n}
\end{equation}
\begin{equation}
P(balanced|data) = \frac{\frac{1}{2^n}}{\frac{1}{2} + \frac{1}{2^n}}
= \frac{1}{1+2^{n-1}}
\end{equation}

We will use this equation to estimate the minimum $n$ required to gain the required confidence. 
\begin{equation}
\frac{1}{1+2^{n-1}} < \epsilon \\
\Rightarrow \frac{1}{\epsilon} < 1 + 2^{n-1} \\ 
\Rightarrow log_{2}(\frac{1}{\epsilon} - 1) + 1 < n \\
\Rightarrow n = ceil(log_{2}(\frac{1}{\epsilon} - 1) + 1)
\end{equation}

In [21]:
def randomised_algo(f, n, acc) : 
    '''
    Input : A function f, number of bits in the input n (natural number)
    Returns : A boolean (true if the function is balanced)
    ''' 
    eps = 1 - acc
    n = int(np.ceil(np.log2(1/eps - 1) + 1))
    header = "0"*n
    num = np.random.randint(1, 2**n)
    num = bin(num).replace("0b", "")
    input = (header + num)[-n:]
    results = [f(input)[-1]]

    for i in range(n-1) : 
    
        num = np.random.randint(1, 2**n)
        num = bin(num).replace("0b", "")
        input = (header + num)[-n:]

        if f(input)[-1] not in results : 
            return True
    
    return False


Now, putting this in action

In [23]:
oracle = balanced_oracle(N)
result = randomised_algo(oracle.draw, N, 0.8)

if result : print("Balanced oracle")
else : print("Constant oracle")

Balanced oracle


In [25]:
oracle = constant_oracle(N)
result = randomised_algo(oracle.draw, N, 0.8)

if result : print("Balanced oracle")
else : print("Constant oracle")

Constant oracle


As we can see, the outputs again match with what is expected.